In [108]:
!pip install pennylane nftopt

In [122]:
#defining parameters
n_qubits=3
depth = 4

In [109]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from scipy import optimize
from nftopt import nakanishi_fujii_todo as nftmethod

In [5]:
dev = qml.device("default.qubit",wires=3)
dev

<default.qubit device (wires=3) at 0x7ac9f9536d70>

In [97]:
def ansatz_layer(layer_weights, depth, n_qubits):
    for dep in range(depth):
    #L1
        for wire in range(n_qubits):
          qml.RY(layer_weights[wire+(n_qubits*dep)], wires=wire)

        qml.CZ(wires=[0,1])
        qml.CZ(wires=[1,2])
        qml.CZ(wires=[2,0])

In [37]:
def state_preparation(x):
    qml.BasisState(x, wires=[0, 1, 2, 3])

In [38]:
def accuracy(labels, predictions):
  state0 = qml.math.dm_from_state_vector(labels)
  state1 = qml.math.dm_from_state_vector(predictions)
  return qml.math.fidelity(state0, state1)

In [39]:
@qml.qnode(dev,diff_method="backprop")
def circuit(weights):
  # since depth 4 was used
  ansatz_layer(weights,depth=depth, n_qubits=n_qubits)
  # qml.state() applies Ua to Ini State, which gives Appro_a
  return qml.state()

In [40]:
def variational_classifier(weights, x):
    # weights are thetas
    return np.real(circuit(weights))

In [41]:
# x = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8])
x = np.array([0.083379922415887,0.049577998492674,0.989090451339352,0.049577998492674,0.049577998492674,0.049577998492674,0.049577998492674,0.049577998492674])
weights = 4*np.pi *np.random.rand(1,12)
print(weights)
predictions_test = variational_classifier(weights, x)
print(predictions_test)
print(accuracy(x,predictions_test))

[[0.44731471 0.75685397 0.20380799 0.2034348  0.50767449 0.1875731
  0.68983928 0.66625399 0.08423187 0.64010809 0.39933108 0.56013735]]
[ 0.00326862  0.20828693  0.77497285 -0.22939319  0.52437271 -0.16354083
 -0.0319628   0.02584106]
0.613606217159294


In [54]:
IniState= np.append(1, np.zeros(2**n_qubits - 1))
print(IniState)

[1. 0. 0. 0. 0. 0. 0. 0.]


In [88]:
def F1_loss(a,Appro_a):
  #   Define the cost function
  F1=(a[0]**2)*np.log2((Appro_a[0]**2)/(a[0]**2))+(a[1]**2)*np.log2((Appro_a[1]**2)/(a[1]**2))+(a[2]**2)*np.log2((Appro_a[2]**2)/(a[2]**2))
  F2=(a[3]**2)*np.log2((Appro_a[3]**2)/(a[3]**2))+(a[4]**2)*np.log2((Appro_a[4]**2)/(a[4]**2))+(a[5]**2)*np.log2((Appro_a[5]**2)/(a[5]**2))
  F3=(a[6]**2)*np.log2((Appro_a[6]**2)/(a[6]**2))+(a[7]**2)*np.log2((Appro_a[7]**2)/(a[7]**2))

  #   Prepare the Bell state
  Bell_State=np.ones((8,1))/np.linalg.norm(np.ones((8,1)))
  #   The sum of vector a
  Sum_a=np.sum(a)
  sqrt_D = np.linalg.norm(np.ones((8,1)))
  UaFunction=np.abs(Sum_a -  (sqrt_D * np.matmul(np.transpose(Bell_State), Appro_a)))-(F1+F2+F3)
  return np.real(UaFunction)

In [83]:
a= x
Appro_a = variational_classifier(weights, a)
F1=((a[0]**2)*np.log2((Appro_a[0]**2)/(a[0]**2))+(a[1]**2)*np.log2((Appro_a[1]**2)/(a[1]**2))+(a[2]**2)*np.log2((Appro_a[2]**2)/(a[2]**2))).numpy()
F2=((a[3]**2)*np.log2((Appro_a[3]**2)/(a[3]**2))+(a[4]**2)*np.log2((Appro_a[4]**2)/(a[4]**2))+(a[5]**2)*np.log2((Appro_a[5]**2)/(a[5]**2))).numpy()
F3=((a[6]**2)*np.log2((Appro_a[6]**2)/(a[6]**2))+(a[7]**2)*np.log2((Appro_a[7]**2)/(a[7]**2))).numpy()
Bell_State=np.ones((8,1))/np.linalg.norm(np.ones((8,1)))
  #   The sum of vector a
# it is plus, but authors made a concept mistake
#Sum_a=np.sum(a)
#UaFunction=np.abs(Sum_a - np.linalg.norm(np.ones((8,1))) * np.transpose(Bell_State) *Appro_a)

2.8284271247461903

In [92]:
def cost(weights, X=a):
    # X is our preparation state
    Appro_a = variational_classifier(weights, X)
    return F1_loss(X, Appro_a)

In [81]:
opt = NesterovMomentumOptimizer(0.5)
np.random.seed(0)

In [64]:
# delta=1;
# Omega=1;
# gamma=1;
# Delta_t=0.01;

# alpha=np.sqrt(2)/2*delta*Delta_t;
# beta=(1-gamma*Delta_t/2);
# aa=[Delta_t*np.sqrt(2)/2*delta,Delta_t*gamma/4,np.sqrt(alpha**2+beta**2),Delta_t*gamma/4,Delta_t*gamma/4,Delta_t*gamma/4,Delta_t*gamma/4,Delta_t*gamma/4]
# x=np.sqrt(aa);
# norma=np.linalg.norm(x);
# x=x/np.linalg.norm(x);

# print(x)

In [119]:
weights = 4*np.pi*np.random.rand(12)
x = np.array([0.083379922415887,0.049577998492674,0.989090451339352,0.049577998492674,0.049577998492674,0.049577998492674,0.049577998492674,0.049577998492674])

# for it in range(1000):
#     # Update the weights by one optimizer step, using only a limited batch of data
#     weights = opt.step(cost, weights, X=x)
#     # Compute accuracy
#     predictions = variational_classifier(weights, x)

#     acc = accuracy(x, predictions)

#     print(f"Iter: {it+1:4d} | Accuracy: {acc:0.7f}")
#     print(weights)
#     print("-----------------------------------------------------------")
result= optimize.minimize(cost,weights, method=nftmethod, options={'maxfev':2000})

In [121]:
predictions=variational_classifier(result.x,a)
acc = accuracy(x, predictions)
print(predictions.numpy(), x.numpy())
acc
#accuracy(a, result.x)

[ 0.11973385 -0.0218794   0.97417981  0.12113409  0.03922353  0.0546745
  0.1229835   0.04280879] [0.08337992 0.049578   0.98909045 0.049578   0.049578   0.049578
 0.049578   0.049578  ]
